In [ ]:
import numpy as np

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Load data

In [ ]:
window_data = np.load('window_data.npy')
unigram_distribution = np.load('unigram_distribution.npy')
word_vectors = np.load('word_vectors.npy')

# Create iterator

In [ ]:
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

class SimpleDataset(Dataset):

    def __init__(self, data_tensor):
        self.data_tensor = data_tensor

    def __getitem__(self, index):
        return self.data_tensor[index]

    def __len__(self):
        return self.data_tensor.size(0)

In [ ]:
batch_size = 128

In [ ]:
dataset = SimpleDataset(torch.FloatTensor(data))

iterator = DataLoader(
    dataset, batch_size=batch_size, num_workers=4,
    shuffle=True, pin_memory=True
)

# number of training samples
data_size = len(data)
data_size

# Model

In [ ]:
# params
n_topics = 20
embedding_dim = 300
vocab_size = 10
window_size = 4
n_documents = 6
num_sampled = 7

In [ ]:
word_vectors = torch.FloatTensor(word_vectors)
unigram_distribution = torch.FloatTensor(unigram_distribution)

In [ ]:
topics = topic_embedding(n_topics, embedding_dim)

In [ ]:
class loss(nn.Module):

    def __init__(self, topics, word_vectors, unigram_distribution):
        super(model, self).__init__()

        doc_embedding = nn.Embedding(n_documents, n_topics)
        doc_embedding.weights = nn.Parameter(2.0*torch.rand(n_documents, n_topics) - 1.0)
        
        self.neg = negative_sampling_loss(word_vectors, unigram_distribution)
        self.topics = topics

    def forward(self, doc_indices, pivot_words, target_words):
        
        alpha = 1.0/n_topics
        lambda_const = 200
        num_sampled = 10
        
        doc_weights = doc_embedding(doc_indices)
        doc_vectors = self.topics(doc_weights)
        
        neg_loss = neg(pivot_words, target_words, doc_vectors, num_sampled)
        dirichlet_loss = lambda_const*(1.0 - alpha)*F.log_softmax(doc_weights).mean(1)

        return neg_loss + dirichlet_loss

In [ ]:
model = loss(topics, word_vectors, unigram_distribution)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True)

In [ ]:
all_losses = []
model.train()
for epoch in range(0, n_epochs):
    for step, batch in enumerate(iterator, 1 + epoch*n_batches):

        batch = Variable(batch.cuda())
        doc_indices = batch[:, 0]
        pivot_words = batch[:, 1]
        target_words = batch[:, 1:]

        total_loss = model(doc_indices, pivot_words, target_words)

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
        all_losses += [total_loss.data[0]]
        print(total_loss.data[0])